In [1]:
#!pip install langchain faiss-cpu sentence-transformers huggingface-hub transformers langchain-community llama-cpp-python

In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = TextLoader("test2.txt",encoding="utf-8")
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = splitter.split_documents(docs)
len(chunks)

d:\final Year project\Research_backend\RAG\venvRAG\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


7

In [3]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

db = FAISS.from_documents(chunks, embeddings)
retriever = db.as_retriever()



C:\Users\mathivathanan\AppData\Local\Temp\ipykernel_26620\1476813871.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")


In [4]:
from langchain_community.llms import LlamaCpp


llm = LlamaCpp(
    model_path=r"C:\Users\mathivathanan\OneDrive - Creative Software (Pvt) Ltd\Desktop\Research project\tamil-llm-fastapi\models\tamil-llama-q4.gguf",
    temperature=0.2,
    max_tokens=256,
    n_ctx=4096,
    verbose=False
)


llama_context: n_batch is less than GGML_KQ_MASK_PAD - increasing to 64


In [5]:
def rag_chain(question):
    # 1. Retrieve similar docs
    docs = retriever.invoke(question)

    # 2. Combine the doc text
    context = "\n\n".join([d.page_content for d in docs])

    # 3. Format the prompt
    prompt_text = f"""
கீழே வழங்கப்பட்டுள்ள 'செய்தி' தவிர வேறு எந்த அறிவையும் பயன்படுத்த வேண்டாம். 
கேள்விக்கான பதில் 'செய்தி' இல் இல்லை என்றால், 'எனக்கு தெரியவில்லை' என்று மட்டும் கூறவும்.

செய்தி:
{context}

கேள்வி: {question}
"""

    # 4. Call the model
    raw = llm.invoke(prompt_text)
    answer = raw.replace(prompt_text, "").strip()
    print(prompt_text)
    return answer

In [6]:
response = rag_chain("நகரமயமாக்கல் வேகமாக வளர்வதற்கான முக்கிய காரணம் என்ன?")
print(response)




கீழே வழங்கப்பட்டுள்ள 'செய்தி' தவிர வேறு எந்த அறிவையும் பயன்படுத்த வேண்டாம். 
கேள்விக்கான பதில் 'செய்தி' இல் இல்லை என்றால், 'எனக்கு தெரியவில்லை' என்று மட்டும் கூறவும்.

செய்தி:
சுற்றுச்சூழல் சமநிலை பாதிக்கப்படுகிறது. இதனை சமாளிக்க அரசு தரப்பில் இருந்து நிலையான நகரமயமாக்கல் (sustainable urbanization) திட்டங்கள் அறிமுகப்படுத்தப்பட்டு வருகின்றன. பொதுப் போக்குவரத்தை மேம்படுத்துதல், மழைநீர் சேகரிப்பு, பசுமை பூங்காக்கள் அமைத்தல் போன்ற நடவடிக்கைகள் மூலம் நகர வாழ்க்கையின் தரத்தை

மற்றும் தொழிற்துறை போன்ற பல துறைகளில் பரவலாக பயன்படுத்தப்பட்டு வருகிறது. மருத்துவ துறையில் நோய்களை முன்னதாக கண்டறிதல், கல்வித் துறையில் மாணவர்களின் கற்றல் திறனை மேம்படுத்துதல், வங்கித் துறையில் மோசடி பரிவர்த்தனைகளை கண்டறிதல் போன்ற பல பயன்கள் AI மூலம் கிடைக்கின்றன. அதே நேரத்தில், வேலை வாய்ப்புகள்

சமீப காலங்களில் நகரமயமாக்கல் வேகமாக வளர்ந்து வருகின்றது. இதன் காரணமாக கிராமப்புறங்களில் இருந்து நகரங்களுக்கு வேலை வாய்ப்புகளை தேடி மக்கள் இடம்பெயர்ந்து வருகின்றனர். நகரங்களில் தொழிற்சாலைகள், தகவல் தொழில்நுட்ப நிறுவனங்கள் மற்ற

In [7]:
response = rag_chain("தமிழ்நாட்டின் தலைநகர் எது?")
print(response)


கீழே வழங்கப்பட்டுள்ள 'செய்தி' தவிர வேறு எந்த அறிவையும் பயன்படுத்த வேண்டாம். 
கேள்விக்கான பதில் 'செய்தி' இல் இல்லை என்றால், 'எனக்கு தெரியவில்லை' என்று மட்டும் கூறவும்.

செய்தி:
சுற்றுச்சூழல் சமநிலை பாதிக்கப்படுகிறது. இதனை சமாளிக்க அரசு தரப்பில் இருந்து நிலையான நகரமயமாக்கல் (sustainable urbanization) திட்டங்கள் அறிமுகப்படுத்தப்பட்டு வருகின்றன. பொதுப் போக்குவரத்தை மேம்படுத்துதல், மழைநீர் சேகரிப்பு, பசுமை பூங்காக்கள் அமைத்தல் போன்ற நடவடிக்கைகள் மூலம் நகர வாழ்க்கையின் தரத்தை

மற்றும் தொழிற்துறை போன்ற பல துறைகளில் பரவலாக பயன்படுத்தப்பட்டு வருகிறது. மருத்துவ துறையில் நோய்களை முன்னதாக கண்டறிதல், கல்வித் துறையில் மாணவர்களின் கற்றல் திறனை மேம்படுத்துதல், வங்கித் துறையில் மோசடி பரிவர்த்தனைகளை கண்டறிதல் போன்ற பல பயன்கள் AI மூலம் கிடைக்கின்றன. அதே நேரத்தில், வேலை வாய்ப்புகள்

கிடைக்கின்றன. அதே நேரத்தில், வேலை வாய்ப்புகள் குறைவதற்கான அச்சம், தனியுரிமை பாதுகாப்பு சவால்கள் மற்றும் தொழில்நுட்பத்தை தவறாக பயன்படுத்தும் அபாயங்கள் போன்ற பிரச்சினைகளும் எழுகின்றன. எனவே, AI தொழில்நுட்பத்தை பொறுப்புடன் மற்றும